In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

# Quick comparison of annual NTD with our data

* note these are not directly comperable for several reasons but in the same ballpark

In [2]:
import numpy as np

In [5]:
vrh_21 = pd.read_excel(f'{shared_utils.rt_utils.GCS_FILE_PATH}ntd_ts21.xlsx', sheet_name='VRH')

In [6]:
vrm_21 = pd.read_excel(f'{shared_utils.rt_utils.GCS_FILE_PATH}ntd_ts21.xlsx', sheet_name='VRM') >> filter(_.State == 'CA')

In [14]:
vrh_21 = vrh_21 >> filter(_.State == 'CA', _.Mode.isin(['MB', 'RB', 'CB', 'TB']))
vrm_21 = vrm_21 >> filter(_.State == 'CA', _.Mode.isin(['MB', 'RB', 'CB', 'TB']))

In [15]:
vrm_21 >> head(3)

,Last Report Year,NTD ID,Legacy NTD ID,Agency Name,Agency Status,Reporter Type,Reporting Module,City,State,Census Year,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4902,2020,90003.0,9003,San Francisco Bay Area Rapid Transit District ...,Active,Full Reporter,Urban,Oakland,CA,2010,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN
4907,2020,90004.0,9004,Golden Empire Transit District (GET),Active,Full Reporter,Urban,Bakersfield,CA,2010,...,3726082.0,3735670.0,3944744.0,3889451.0,3416623.0,3848798.0,3914833.0,3902753.0,3885910.0,3634980.0
4910,2020,90006.0,9006,Santa Cruz Metropolitan Transit District (SCMTD),Active,Full Reporter,Urban,Santa Cruz,CA,2010,...,NaN,614838.0,610983.0,683000.0,683260.0,686891.0,611806.0,603288.0,606296.0,500933.0


In [31]:
vrm_to_sum = vrm_21.iloc[:,27:]

In [32]:
vrh_to_sum = vrh_21.iloc[:,27:]

In [33]:
vrm_to_sum.sum() / vrh_to_sum.sum()

2000    12.744248
2001    12.687300
2002    12.596026
2003    12.442103
2004    12.488847
2005    12.375109
2006    12.377809
2007    12.432010
2008    12.315688
2009    12.248419
2010    12.223065
2011    12.103078
2012    12.028518
2013    12.023532
2014    11.961174
2015    11.924564
2016    11.903092
2017    11.725700
2018    11.657684
2019    11.688012
2020    11.665352
dtype: float64

In [34]:
analysis_date = dt.date(2022, 6, 1) ##for exec board (prior run for comparison)

In [35]:
day = str(analysis_date.day).zfill(2)
month = str(analysis_date.month).zfill(2)

In [37]:
import gcsfs
fs = gcsfs.GCSFileSystem()

In [38]:
fs_list = fs.ls(f'{shared_utils.rt_utils.GCS_FILE_PATH}rt_trips/')

In [48]:
## now finds ran operators on specific analysis date
ran_operators = ['gs://' + path for path in fs_list
                 if path.split('rt_trips/')[1] and path.split('rt_trips/')[1].split('_')[1] == month and path.split('rt_trips/')[1].split('_')[2][:2] == day]

In [49]:
ran_operators

['gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/110_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/126_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/127_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/135_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/148_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/159_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/167_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/170_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/182_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/183_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/188_06_01.parquet',
 'gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/194_06_01.parquet',
 'gs://calitp-analytics-data

In [51]:
all_rt_trips = pd.DataFrame()
for path in ran_operators:
    all_rt_trips = pd.concat([pd.read_parquet(path), all_rt_trips])

In [55]:
all_rt_trips >> filter(_.route_type == '3') >> summarize(mean_all = _.mean_speed_mph.mean())

,mean_all
0,12.590907


In [56]:
pd.read_parquet('gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/282_06_01.parquet') >> filter(_.route_type == '3') >> summarize(mean_all = _.mean_speed_mph.mean())

,mean_all
0,10.186188


In [57]:
pd.read_parquet('gs://calitp-analytics-data/data-analyses/rt_delay/rt_trips/182_06_01.parquet') >> filter(_.route_type == '3') >> summarize(mean_all = _.mean_speed_mph.mean())

,mean_all
0,11.792687
